In [3]:
import pandas as pd 
import numpy as np
from collections import namedtuple
# setup

columns_filter = [
    "date",
    "LD_ID",
    "loadId",
    "UT_ID",
    "unit",
    "VDR_SHP_ID",
    "MPRS PO",
    "purchaseOrder",
    "MPRS GRSS CST",
    "grossCost",
    "MPRS NET CST",
    "netCost",
    "MPRS LIST CST",
    "meijerListCost",
    "MPRS VDRLIST",
    "vendorList",
    "MPRS WGT RCV",
    "receiveWeight",
]
column_map = namedtuple("ColMap", "mprs_col core_col diff_col")
columns_mprs_core = [
    column_map("MPRS GRSS CST", "grossCost", "diff_grossCost"),
    column_map("MPRS NET CST", "netCost", "diff_netCost"),
    column_map("MPRS LIST CST", "meijerListCost", "diff_meijerListCost"),
    column_map("MPRS VDRLIST", "vendorList", "diff_vendorList"),
    column_map("MPRS WGT RCV", "receiveWeight", "diff_receiveWeight"),
]

# Identify the mis-matches and difference
def find_mismtach_summary(df):
    summarized_data = {}
    for col in columns_mprs_core:
        summarized_data[col.diff_col] = df[col.mprs_col] - df[col.core_col]
    df1 = pd.concat(summarized_data, axis=1)
    return df1

# Read Excel Sheet
df = pd.read_excel("data.xlsx", engine="openpyxl", usecols=columns_filter)
df = df.fillna(0)

df1 = find_mismtach_summary(df)
#Merge the data set with identified difference
df2 = (pd.concat([df, df1], axis=1))
filter_df = df2[(df2['diff_grossCost'] != 0)
                | (df2['diff_netCost']!=0)
                | (df2['diff_meijerListCost'] != 0)
                | (df2['diff_vendorList']!=0)
                | (df2['diff_receiveWeight'] != 0)]

filter_df.to_excel("mismatch1.xlsx")

c:\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# column name list  
# Define a method to calculate the percentage of each column greater than 5 and less than 10
def percentage_between (df):
    col_names =  ['Name','Penny','PennyPercentage','Mismatch','MismatchPercentage','Equal','EqualPercentage']   
    analytical_df  = pd.DataFrame(columns = col_names) 
    colNames = df.columns.tolist()
    for colName in colNames:
        # Count the number of values between low and high in each column
        pennydifference = df[colName].between(0.000001,0.999999).sum() + df[colName].between(-0.999999,-0.000001).sum()       
        pennydifferencePercentage = pennydifference / len(df) * 100
        totalmismatch = df[colName].gt(0.99999).sum() + df[colName].lt(-0.99999).sum()
        totalmismatchPercentage = totalmismatch / len(df) * 100
        totalequal = df[colName].eq(0).sum()
        totalequalPercentage = totalequal / len(df) * 100
        # New list for append into df
        list = [colName, pennydifference,pennydifferencePercentage,totalmismatch ,totalmismatchPercentage,totalequal ,totalequalPercentage]
        analytical_df.loc[len(analytical_df)] = list
        
    # Return the percentage as a data frame  
    return analytical_df
# Apply the function to the data frame with different thresholds
df_percentage = percentage_between (df1)
print(df_percentage)



                  Name  Penny  PennyPercentage  Mismatch  MismatchPercentage  \
0       diff_grossCost     19         0.014745       362            0.280925   
1         diff_netCost   2991         2.321124      7153            5.550986   
2  diff_meijerListCost     61         0.047338      2591            2.010709   
3      diff_vendorList     47         0.036474      2627            2.038647   
4   diff_receiveWeight   3605         2.797610      3852            2.989291   

    Equal  EqualPercentage  
0  128479        99.704330  
1  118716        92.127891  
2  126208        97.941953  
3  126186        97.924880  
4  121403        94.213099  
